In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from config import *

import scipy.io
# relevent_columns = dep_var1 + dep_var2 + ['刀盘功率']
gpu_start,num_gpus = 2,1

In [ ]:
test1_fns = [Path('tbmData/test/附件2：必选课题二测试数据集') / f'{i}.mat' for i in range(51, 101)]
column_names = pd.read_csv('tbmData/data/CREC188_20160809.txt', delimiter='\t', nrows=1).columns # get column names from training data
save_dir = Path('tbmData/test/test1_feathers')

In [ ]:
test1_fns[:5]

# Save test data as feathers

Assert the last column of test data is all zero

In [ ]:
column_names

In [ ]:
def read_test_mat_as_df(mat_fn, columns):
    test_input = scipy.io.loadmat(mat_fn)['a']
    test_df = pd.DataFrame(test_input[:,:-1], columns=columns[2:])
    return test_df

for mat_fn in test1_fns:
    df = read_test_mat_as_df(mat_fn, column_names)
    save_feather_fn = save_dir/(mat_fn.stem + '.feather')
    df.to_feather(str(save_feather_fn))

In [ ]:
context = Context('save-after-removing-redundent-columns', fn_cycles=str(save_dir), debug=0, mulr=1, load_data=False, gpu_start=)
fmtr = DataFormatter(context, cycle_feathers=context.fn_cycles)
cycles = fmtr.cycles

In [ ]:
dropped = np.load('tbmData/cycles-removed-redundent-columns/dropped.npy').tolist()
cycles.drop(columns=dropped+['桩号']+['EP2次数设置', '刀盘喷水增压泵压力'], inplace=True, errors='ignore')
# , '内循环水罐液位', '拖拉油缸最小允许压力设置'
cycles.loc[:,'总推进力'].mean()

In [ ]:
context = Context('amended-3-7', gpu_start=gpu_start, dep_var=dep_var1, debug=0, mulr=1)
stat_x = context.stat_x[:,:-1]
stat_extra_x = context.stat_extra_x
valid_size = int(context.num_cycles * context.valid_ratio * context.mulr)
databunch = MultiDeptTabularDataBunch.from_df('tmp', context.cyc_cont, context.dep_var, valid_idx=context.valid_idx_tile, bs=valid_size)

In [ ]:
normalized_x = normalize_a(cycles, *stat_x)
xb = [tensor(normalized_x.loc[i].values.flatten())[None].float() for i in normalized_x.index.levels[0]]
xb = torch.cat(xb)
xb.shape

In [ ]:
# append extra_x
csv_extra = pd.read_csv(u'tbmData/test/附件2 测试数据（第51-100组数据稳态段操作参数值，含桩号）.csv',encoding='gbk')
extra_x_normalized = normalize_a(csv_extra.iloc[:,1:], *stat_extra_x)
extra_x_normalized_t = tensor(extra_x_normalized.values)
xb = torch.cat([xb, extra_x_normalized_t.float()], 1)
xb.shape

# predict and submit

In [ ]:
n_cont = 2
n_cat = [5, 6]
cat_names = ['围岩等级', '岩性']

hyper_params = {
    'use_extra_x': 1,
    'n_hidden': 1000,
    'n_layers': 3,
    'hidden_p': 0.075,
    'input_p': 0.1,
    'weight_p': 0.125,
    'drops': [0.1] * 4,
    'alpha': 2.,
    'beta': 1.,
    'clip': 0.5,
    'loss_func': DualLoss(n_cont, n_cat, 0.1)
}

hyper_params['layers'] = [3*(context.n_cont) + int(hyper_params['use_extra_x']) * 3, 1000, 13]
learner = get_new_model(context, databunch, hyper_params)

In [ ]:
# learner.load('400_amended-3-7_2019-04-18_23:05:30');
learner.load('final-no-extra-day_1000_amended-3-7-cat_2019-04-29_22:18:11');

model = learner.model
model.eval()
model.reset()
input = (torch.zeros(len(xb), dtype=torch.long).cuda(), xb.cuda())
pb = model(*input)
pb = denormalize(pb[0][:,:2], *context.stat_y)
pb = to_np(pb)

In [ ]:
output_template = 'tbmData/test/附件3 测试结果提交格式.csv'
template = pd.read_csv(output_template,encoding='gbk')

template.iloc[:,1:3] = pb
template

In [ ]:
submit_dir = Path('submissions/submission-4-30')
submit_dir.mkdir(parents=True,exist_ok=True)
template.to_csv(str(submit_dir/'Result_test_51-100.csv'), float_format='%.1f',index=None, encoding='gbk')

# validate extra x statistics

In [ ]:
from IPython.display import HTML
def series_to_html(series, title=None):
    return HTML(pd.DataFrame(series, columns=[title]).to_html())
s2h = series_to_html

In [ ]:
# our statistics
our_extra_x = cycles[dep_var1]
# display(s2h(our_extra_x.mean(), 'mean'), s2h(our_extra_x.std(), 'std'))
pd.DataFrame(
    {
        'mean': our_extra_x.mean(), 
        'std': our_extra_x.std()
    }
)

In [ ]:
# their statistics
their_extra_x = csv_extra.iloc[:,2:]
# display(s2h(their_extra_x.mean(), 'mean'), s2h(their_extra_x.std(), 'std'))
pd.DataFrame(
    {
        'mean': their_extra_x.mean(), 
        'std': their_extra_x.std()
    }
)

In [ ]:
sns.set(color_codes=True)

In [ ]:
def plot_dist(df):
    fig = plt.figure(figsize=(15,10))
    for v in df:
        sns.distplot(df.loc[:,v], hist=False, rug=True);
    ax = fig.gca()
    ax.legend(df.columns)
    ax.set_xlabel('value')
    ax.set_ylabel('p')
    set_ax_font(ax)
plot_dist(their_extra_x)
plt.xlim([0,100])
plot_dist(our_extra_x)
plt.xlim([0,100]);

# validate prediction code

In [ ]:
context = Context('amended-3-7', debug=0, dep_var=dep_var2, mulr=7, gpu_start=gpu_start)
databunch = MultiDeptTabularDataBunch.from_df('tmp', context.cyc_cont, context.dep_var, valid_idx=context.valid_idx_tile, bs=context.bs)

In [ ]:
context.valid_idx_tile.sort()
context.valid_idx_tile

In [ ]:
plt.plot(context.valid_idx_tile)

In [ ]:
x_valid, y_valid = ni(databunch.valid_dl)

In [ ]:
y_valid

In [ ]:
x_valid[1].shape

In [ ]:
# load training and validation data
# context = Context('amended-1-no_noise', debug=0, dep_var=dep_var2, mulr=1, gpu_start=9)
context = Context('amended-3-7', debug=0, dep_var=dep_var2, mulr=7, gpu_start=gpu_start)
databunch = MultiDeptTabularDataBunch.from_df('tmp', context.cyc_cont, context.dep_var, valid_idx=context.valid_idx_tile, bs=context.bs)

In [ ]:
xb, yb = ni(databunch.train_dl)
xb = xb[1]
yb = to_np(yb)

xb_valid, yb_valid = ni(databunch.valid_dl)
xb_valid, yb_valid = xb_valid[1], to_np(yb_valid)

In [ ]:
# learner.load('400_amended-3-7_2019-04-18_23:05:30');
learner.load('best_lstm_1000_amended-3-7_2019-04-27_22:13:46', purge=False);

model = learner.model
model.eval()
model.reset()
input = (torch.zeros(len(xb), dtype=torch.long).cuda(), xb.cuda())
pb = model(*input)
input = (torch.zeros(len(xb_valid), dtype=torch.long).cuda(), xb_valid.cuda())
pb_valid = model(*input)

# pb = denormalize(pb[0], *context.stat_y)
pb = pb[0]
pb = to_np(pb)
pb_valid = pb_valid[0]
pb_valid = to_np(pb_valid)

In [ ]:
df_scores = pd.DataFrame({
    'Train MAPD': [context.metrics(tensor(pb), tensor(yb)).item()],
    'Valid MAPD': [context.metrics(tensor(pb_valid), tensor(yb_valid)).item()],
})
df_scores

In [ ]:
mean = torch.zeros_like(tensor(pb_valid)).float()
# input
df_scores = pd.DataFrame({
    'Train MAPD': [context.metrics(tensor(pb), tensor(yb)).item()],
    'Valid MAPD': [context.metrics(tensor(pb_valid), tensor(yb_valid)).item()],
    'Mean MAPD': [context.metrics(mean, tensor(yb_valid)).item()],
})
df_scores

In [ ]:
i = 0
# yb,pb = target,input
# (np.abs(yb[:,i] - pb[:,i]) / yb[:,i]).mean()
(np.abs(yb_valid - pb_valid) / yb_valid).mean()

In [ ]:
# Now let's quickly train a random forest with various kinds of datasets
from sklearn.ensemble import forest
from sklearn.ensemble import RandomForestRegressor
def set_rf_samples(n):
    """ Changes Scikit learn's random forests to give each tree a random sample of
    n random rows.
    """
    forest._generate_sample_indices = (lambda rs, n_samples:
        forest.check_random_state(rs).randint(0, n_samples, n))

def reset_rf_samples():
    """ Undoes the changes produced by set_rf_samples.
    """
    forest._generate_sample_indices = (lambda rs, n_samples:
        forest.check_random_state(rs).randint(0, n_samples, n_samples))
def print_score(m):
    res = [m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)
    
def rf_feat_importance(m, df):
    return pd.DataFrame({'cols':df.columns, 'imp':m.feature_importances_}
                       ).sort_values('imp', ascending=False)

In [ ]:
# set_rf_samples(100)
reset_rf_samples()

In [ ]:
# load training and validation data
# context = Context('amended-1-no_noise', debug=0, dep_var=dep_var2, mulr=1, gpu_start=9)
context = Context('amended-3-7', debug=0, dep_var=dep_var2, mulr=7, gpu_start=gpu_start)
databunch = MultiDeptTabularDataBunch.from_df('tmp', context.cyc_cont, context.dep_var, valid_idx=context.valid_idx_tile, bs=context.bs)

In [ ]:
xb, yb = ni(databunch.train_dl)
X_train, y_train = to_np(xb[1]), to_np(yb)

xb, yb = ni(databunch.valid_dl)
X_valid, y_valid = to_np(xb[1]), to_np(yb)

In [ ]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=80, oob_score=True, min_samples_leaf=1)
m.fit(X_train, y_train)
df_scores = pd.DataFrame({
    'Train score': [m.score(X_train, y_train)],
    'Valid score': [m.score(X_valid, y_valid)],
    'OOB': [m.oob_score_]
})
df_scores

In [ ]:
# m = RandomForestRegressor(n_jobs=-1, n_estimators=80, oob_score=True, min_samples_leaf=5)
m = RandomForestRegressor(n_jobs=-1, n_estimators=40, oob_score=True, min_samples_leaf=3, max_features=0.2)
m.fit(X_train, y_train)

p_train, p_valid = m.predict(X_train), m.predict(X_valid)
# random forest
df_scores = pd.DataFrame({
    'Train MAPD': [context.metrics(torch.tensor(p_train, dtype=torch.float), torch.tensor(y_train, dtype=torch.float)).item()],
    'Valid MAPD': [context.metrics(torch.tensor(p_valid, dtype=torch.float), torch.tensor(y_valid, dtype=torch.float)).item()],
    'Train score': [m.score(X_train, y_train)],
    'Valid score': [m.score(X_valid, y_valid)],
    'OOB': [m.oob_score_],
})
df_scores

In [ ]:
# m = RandomForestRegressor(n_jobs=-1, n_estimators=80, oob_score=True, min_samples_leaf=5)
m = RandomForestRegressor(n_jobs=-1, n_estimators=80, oob_score=True, min_samples_leaf=3, max_features=0.2)
m.fit(X_train, y_train)

p_train, p_valid = m.predict(X_train), m.predict(X_valid)
# random forest
df_scores = pd.DataFrame({
    'Train MAPD': [context.metrics(torch.tensor(p_train, dtype=torch.float), torch.tensor(y_train, dtype=torch.float)).item()],
    'Valid MAPD': [context.metrics(torch.tensor(p_valid, dtype=torch.float), torch.tensor(y_valid, dtype=torch.float)).item()],
    'Train score': [m.score(X_train, y_train)],
    'Valid score': [m.score(X_valid, y_valid)],
    'OOB': [m.oob_score_],
})
df_scores

In [ ]:
# m = RandomForestRegressor(n_jobs=-1, n_estimators=80, oob_score=True, min_samples_leaf=5)
m = RandomForestRegressor(n_jobs=-1, n_estimators=80, oob_score=True, min_samples_leaf=3, max_features=0.5)
m.fit(X_train, y_train)

p_train, p_valid = m.predict(X_train), m.predict(X_valid)
# random forest
df_scores = pd.DataFrame({
    'Train MAPD': [context.metrics(torch.tensor(p_train, dtype=torch.float), torch.tensor(y_train, dtype=torch.float)).item()],
    'Valid MAPD': [context.metrics(torch.tensor(p_valid, dtype=torch.float), torch.tensor(y_valid, dtype=torch.float)).item()],
    'Train score': [m.score(X_train, y_train)],
    'Valid score': [m.score(X_valid, y_valid)],
    'OOB': [m.oob_score_],
})
df_scores

In [ ]:
# load training and validation data
# context = Context('amended-1-no_noise', debug=0, dep_var=dep_var2, mulr=1, gpu_start=9)
context = Context('amended-3-less-features-7', debug=0, dep_var=dep_var2, mulr=7, gpu_start=gpu_start)
databunch = MultiDeptTabularDataBunch.from_df('tmp', context.cyc_cont, context.dep_var, valid_idx=context.valid_idx_tile, bs=context.bs)

In [ ]:
# m = RandomForestRegressor(n_jobs=-1, n_estimators=80, oob_score=True, min_samples_leaf=5)
m = RandomForestRegressor(n_jobs=-1, n_estimators=80, oob_score=True, min_samples_leaf=3, max_features=0.5)
m.fit(X_train, y_train)

p_train, p_valid = m.predict(X_train), m.predict(X_valid)
# random forest
df_scores = pd.DataFrame({
    'Train MAPD': [context.metrics(torch.tensor(p_train, dtype=torch.float), torch.tensor(y_train, dtype=torch.float)).item()],
    'Valid MAPD': [context.metrics(torch.tensor(p_valid, dtype=torch.float), torch.tensor(y_valid, dtype=torch.float)).item()],
    'Train score': [m.score(X_train, y_train)],
    'Valid score': [m.score(X_valid, y_valid)],
    'OOB': [m.oob_score_],
})
df_scores

In [ ]:
p_valid.shape

In [ ]:
y_true = denormalize(y_valid, *context.stat_y)
y_lstm = denormalize(pb_valid, *context.stat_y)
y_rf = denormalize(p_valid, *context.stat_y)

In [ ]:
i = 1
(np.abs(y_true[:,i] - y_lstm[:,i]) / y_true[:,i]).mean()

In [ ]:
np.mean()

In [ ]:
df_valid_result = pd.DataFrame({
    'F_true': y_true[:,0],
    'T_true': y_true[:,1],
    'F_lstm': y_lstm[:,0],
    'T_lstm': y_lstm[:,1],
    'F_rf': y_rf[:,0],
    'T_rf': y_rf[:,1],
})

In [ ]:
df_valid_result.to_csv('plots/pred.csv')

# submit random forest result


In [ ]:
xb_test = to_np(xb)

In [ ]:
pb = m.predict(xb_test)

pb = denormalize(pb, *context.stat_y)

In [ ]:
output_template = 'tbmData/test/附件3 测试结果提交格式.csv'
template = pd.read_csv(output_template,encoding='gbk')

template.iloc[:,1:3] = pb
template

In [ ]:
submit_dir = Path('submissions/submission-4-26')
submit_dir.mkdir(parents=True,exist_ok=True)
template.to_csv(str(submit_dir/'Result_test_51-100.csv'), float_format='%.1f',index=None, encoding='gbk')

In [ ]:
display_all(rf_feat_importance(m, context.cyc_cont.iloc[:,:-2]))